# Prepare of weather data

Data set contains Weather data for Kiel from January 1, 2012 to August 1, 2019.

    Datum : gives the date of the observed weather.
    Bewoelkung : describes the cloud coverage. Check this site for more info on the values.
    Temperature : gives the temperature in Celsius.
    Windgeschwindigkeit : gives the wind speed in meters per second.
    Wettercode : describes the type of weather. Check this site for more info on the values.

In [1]:
import pandas as pd

# load the weather dataset
weather_data = pd.read_csv('../bakery_sales_data/wetter.csv')

# change date format before merge to have same datatype on key
weather_data['Datum'] = pd.to_datetime(weather_data['Datum'])

# normalize range to base dataset
base_dateranges = pd.read_pickle('../exported_data/base_dateranges_data.pkl')

prepared_weather_data =  base_dateranges.merge(weather_data, on='Datum', how='left')

## Investigate date and find missing values


## 

## Calculate and group more columns

- monthly_mean_temp_diff : The deviation of the daily temperature from the monthly mean temperature in degeree C 
- TODO: more to add!!!


In [3]:
# calculate and add mean temperature per month and deviation per day
# extract month and year
prepared_weather_data['Jahr_Monat'] = prepared_weather_data['Datum'].dt.to_period('M')

# calulate mean temperature per month
monthly_mean_temp = prepared_weather_data.groupby('Jahr_Monat')['Temperatur'].transform('mean')

# add the mean temperature as extra column
prepared_weather_data['monthly_mean_temp'] = monthly_mean_temp

# calculate the diff from month mean temp to temperature on day
prepared_weather_data['monthly_mean_temp_diff'] = prepared_weather_data['Temperatur'] - prepared_weather_data['monthly_mean_temp']

# store the prepared dataframe
prepared_weather_data.to_pickle('../exported_data/prepared_weather_data.pkl')